# Data Collection for Potential Features

---
This notebook reads in the various files that I collected, mainly from government websites, that contain data that might be interesting features in a model to explain / predict US population mortality changes over time.

---

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.style.use('wright')

#### Create a new DataFrame to systematically store the information that will be loaded.

In [2]:
# Create DF to store features
X = pd.DataFrame({
    'year':range(1900, 2019),
})

#### Read in GDP data gathered from FRED database (Federal Reserve Bank of St. Louis)

In [3]:
# Read in GDP data 

def extract_year(val):
    out = int(val[-2:])+1900
    if out < 1947:
        out+=100
    return(out)

df = pd.read_csv('../data/GDP.csv')
df['year'] = df['DATE'].map(extract_year)
df2 = df.groupby('year').sum()
X = pd.merge(X, df2, on = ['year'], how = 'left')

#### Read in the income inequality file
---

This is a file that I created by taking the difference between the 40% percentile income and the 95% percentile income.  Source: U.S. Census Bureau, Current Population Survey, Annual Social and Economic Supplements.  Table H-1 Income Limits for Each Fifth and Top 5 Percent of All Households.

---

In [4]:
df = pd.read_csv('../data/inc_ineq.csv')
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Read in labor force participation rates
---

From the Bureau of Labor Statistics  
Labor Force Statistics from teh Current Population Survey . 
Series ID:  LNS 11300000  
Years 1948 to 2018

---

In [5]:
df = pd.read_csv('../data/labor_participation_rate_us_1948to2018.csv', skiprows=11)
col = list(df.columns)
col[0]='year'
col[6]='lab_par_rate'
df.columns = col
df.head()

X = pd.merge(X, df[['year','lab_par_rate']], on = ['year'], how = 'left')

#### Read in military spending data
----

Source:  World Bank's databased of World Development Indicators

---

In [6]:
df = pd.read_csv('../data/military_spending.csv', skiprows=4)
df2 = df[df['Country Name'] == 'United States'].copy()

df3 = pd.DataFrame({
    'year': range(1960,2018),
    'military_spend':list(df2.unstack()[4:62])
})

X = pd.merge(X, df3, on = ['year'], how = 'left')


In [7]:
df = pd.read_csv('../data/military_spending_perc_gdp.csv', skiprows=4)
df2 = df[df['Country Name'] == 'United States'].copy()

df3 = pd.DataFrame({
    'year': range(1960,2018),
    'military_spend_pct_gdp':list(df2.unstack()[4:62])
})

X = pd.merge(X, df3, on = ['year'], how = 'left')


In [8]:
df = pd.read_csv('../data/militaryspend_percap.csv')
df.drop(['military_spend_curr_USD','pop_est_UN'], axis = 1, inplace = True)
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Read in inflation data
----

Source:  World Bank's databased of World Development Indicators

---

In [9]:
df = pd.read_csv('../data/inflation.csv', skiprows=4)
df2 = df[df['Country Name'] == 'United States'].copy()

df3 = pd.DataFrame({
    'year': range(1960,2018),
    'inflation':list(df2.unstack()[4:62])
})

X = pd.merge(X, df3, on = ['year'], how = 'left')

#### Read in historical returns on the S&P 500, 3 month UST, and 10 year UST
---
Data taken from NYU Stern school of business website:  
http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html  

---

In [10]:
df = pd.read_csv('../data/fin_market_data_28to17.csv', usecols=range(0,4))
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Read in inflation data
----

Source:  World Bank's databased of World Development Indicators
Rate of increase for 1960 estimated from data online.   

---

In [11]:
df = pd.read_csv('../data/gdp_percap.csv')
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Read in proportion of US population covered by health insurance
---

Regardless of source (ie, public or private) . 
Combination of data sources:  
> 2007 - 2017 US Census Bureau's Annual Social & Economic Supplement  
> 1992 - 2006 CDC / NIH  
> variety of sources prior to 1992  
> linear interpolation for missing values



In [12]:
df = pd.read_csv('../data/ins_pop.csv')
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Obesity 
---
Obesity statistics split between overweight, obese, and severly obese.  
Note severely obese are included in the obese figures.  
Source:  CDC

---

In [13]:
df = pd.read_csv('../data/obesity_from_cdc.csv',usecols=range(0,4))
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Alocohol consumption
---

>Source:  National Institute of Alcohol Abuse and Alcoholism Surveillance Report #108, April 2017  
>Gallons of ethanol consumed per capita in the US by year and type of alcoholic beverage.  

---

In [14]:
df = pd.read_csv('../data/alcohol_to2015.csv', header=2)
df.drop(['beer','wine','spirits'], axis = 1, inplace = True)
df.columns = ['year','alc_per_capita']
X = pd.merge(X, df, on = ['year'], how = 'left')

#### Smoking 
---
> Percentage of the population that smokes tobacco.  
> From CDC survey 2016 from https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm  
> Missing values linearly interpolated

---

In [15]:
df = pd.read_csv('../data/smoking_to2016.csv', header=4)
X = pd.merge(X, df, on = ['year'], how = 'left')

In [16]:
X.head()

year  GDP  hhi_40ptl  hhi_95ptl  hhi_ineq  lab_par_rate  military_spend  \
0  1900  NaN        NaN        NaN       NaN           NaN             NaN   
1  1901  NaN        NaN        NaN       NaN           NaN             NaN   
2  1902  NaN        NaN        NaN       NaN           NaN             NaN   
3  1903  NaN        NaN        NaN       NaN           NaN             NaN   
4  1904  NaN        NaN        NaN       NaN           NaN             NaN   

   military_spend_pct_gdp  military_spend_per_capita  perc_inc_mil_spend  \
0                     NaN                        NaN                 NaN   
1                     NaN                        NaN                 NaN   
2                     NaN                        NaN                 NaN   
3                     NaN                        NaN                 NaN   
4                     NaN                        NaN                 NaN   

        ...        Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  ins_rate  \
0       ...               NaN         NaN         NaN         NaN       NaN   
1       ...               NaN         NaN         NaN         NaN       NaN   
2       ...               NaN         NaN         NaN         NaN       NaN   
3       ...               NaN         NaN         NaN         NaN       NaN   
4       ...               NaN         NaN         NaN         NaN       NaN   

   overweight  obese  severe_obese  alc_per_capita  adult_tob_perc  
0         NaN    NaN           NaN             NaN             NaN  
1         NaN    NaN           NaN             NaN             NaN  
2         NaN    NaN           NaN             NaN             NaN  
3         NaN    NaN           NaN             NaN             NaN  
4         NaN    NaN           NaN             NaN             NaN  

[5 rows x 26 columns]

In [17]:
X.tail()

year        GDP  hhi_40ptl  hhi_95ptl  hhi_ineq  lab_par_rate  \
114  2014  70086.988    42688.0   214100.0    5.0155          62.8   
115  2015  72877.186    45020.0   221900.0    4.9289          62.7   
116  2016  74828.755    46581.0   230095.0    4.9397          62.7   
117  2017  77941.576    47110.0   237034.0    5.0315          62.8   
118  2018  40452.971        NaN        NaN       NaN          62.9   

     military_spend  military_spend_pct_gdp  military_spend_per_capita  \
114    6.099140e+11                3.499702                     1915.0   
115    5.961050e+11                3.289634                     1858.0   
116    6.001060e+11                3.222135                     1857.0   
117    6.097580e+11                3.149222                        NaN   
118             NaN                     NaN                        NaN   

     perc_inc_mil_spend       ...        Unnamed: 3  Unnamed: 4  Unnamed: 5  \
114             -0.0536       ...               NaN         NaN         NaN   
115             -0.0297       ...               NaN         NaN         NaN   
116             -0.0002       ...               NaN         NaN         NaN   
117                 NaN       ...               NaN         NaN         NaN   
118                 NaN       ...               NaN         NaN         NaN   

     Unnamed: 6  ins_rate  overweight  obese  severe_obese  alc_per_capita  \
114         NaN      88.5        31.9   70.1           8.1            2.32   
115         NaN      90.9        31.0   71.0           8.0            2.32   
116         NaN      91.2        31.0   71.0           8.0             NaN   
117         NaN      90.9         NaN    NaN           NaN             NaN   
118         NaN       NaN         NaN    NaN           NaN             NaN   

     adult_tob_perc  
114            16.8  
115            16.7  
116            16.5  
117             NaN  
118             NaN  

[5 rows x 26 columns]

In [18]:
X.to_csv('../data/features.csv')